In [1]:
import os
import pandas as pd
from datetime import datetime
from collections import Counter
import hail as hl
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd

Loading BokehJS ...

In [2]:
auxiliary_path = "gs://fc-aou-datasets-controlled/v7/wgs/short_read/snpindel/aux"
ancestry_path = f'{auxiliary_path}/ancestry'
ancestry_pred_path = f'{ancestry_path}/ancestry_preds.tsv'
ancestry_pred = hl.import_table(ancestry_pred_path,
                               key="research_id", 
                               impute=True, 
                               types={"research_id":"tstr","pca_features":hl.tarray(hl.tfloat)})

Initializing Hail with default parameters...
/opt/conda/lib/python3.10/site-packages/hailtop/aiocloud/aiogoogle/user_config.py:43: UserWarning:

Reading spark-defaults.conf to determine GCS requester pays configuration. This is deprecated. Please use `hailctl config set gcs_requester_pays/project` and `hailctl config set gcs_requester_pays/buckets`.

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
Running on Apache Spark version 3.3.0
SparkUI available at http://all-of-us-9013-m.c.terra-vpc-sc-30761929.internal:41543
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.130-bea04d9c79b5
LOGGING: writing to /home/jupyter/workspaces/crc/hail-20240614-2159-0.2.130-bea04d9c79b5.log
2024-06-14 22:00:15.691 Hail: INFO: Reading table to impute column types 1) / 1]
2024-06-14 22:00:19.868 Hail: INFO: Finished type imputation        (0 + 1) / 1]
  Loading field '

In [3]:
import hail as hl
import os
bucket = os.getenv("WORKSPACE_BUCKET")

hl.init(default_reference='GRCh38', idempotent=True)
save_path = f'{bucket}/data/.mt'
mt = hl.read_matrix_table(save_path)

In [4]:
sample_ids = mt.s.collect()

In [8]:
save_path = f'{bucket}/data/mlh1_0.csv'
g1 = pd.read_csv(save_path)
for i in range(4):
    g1 = pd.concat([g1, pd.read_csv(f'{bucket}/data/mlh1_'+str(i+1)+'.csv')])
print(len(g1['pat'].tolist()))
print(len(set(g1['pat'].tolist())))
mlh1_pat = list(set(g1['pat'].tolist()))

67
67


In [9]:
save_path = f'{bucket}/data/msh2_0.csv'
g2 = pd.read_csv(save_path)
for i in range(4):
    g2 = pd.concat([g2, pd.read_csv(f'{bucket}/data/msh2_'+str(i+1)+'.csv')])
print(len(g2['pat'].tolist()))
print(len(set(g2['pat'].tolist())))
msh2_pat = list(set(g2['pat'].tolist()))

60
60


In [10]:
save_path = f'{bucket}/data/msh6_0.csv'
g3 = pd.read_csv(save_path)
for i in range(4):
    g3 = pd.concat([g3, pd.read_csv(f'{bucket}/data/msh6_'+str(i+1)+'.csv')])
print(len(g3['pat'].tolist()))
print(len(set(g3['pat'].tolist())))
msh6_pat = list(set(g3['pat'].tolist()))

225
225


In [11]:
save_path = f'{bucket}/data/pms2_0.csv'
g4 = pd.read_csv(save_path)
for i in range(1):
    g4 = pd.concat([g4, pd.read_csv(f'{bucket}/data/pms2_'+str(i+1)+'.csv')])
print(len(g4['pat'].tolist()))
print(len(set(g4['pat'].tolist())))
pms2_pat = list(set(g4['pat'].tolist()))

331
331


In [14]:
ancestry_pred = ancestry_pred.filter(hl.literal(list(set(sample_ids))).contains(ancestry_pred['research_id']))
ancestry_pred_df = ancestry_pred.to_pandas()
ancestry_pred_df = ancestry_pred_df.rename(columns = {'research_id':'person_id'})
ancestry_pred_df = ancestry_pred_df[['person_id','ancestry_pred']]

2024-06-14 18:48:43.709 Hail: INFO: Coerced sorted dataset          (0 + 1) / 1]


In [12]:
dataset_15423695_person_sql = """
    SELECT
        person.person_id,
        person.gender_concept_id,
        p_gender_concept.concept_name as gender,
        person.birth_datetime as date_of_birth,
        person.race_concept_id,
        p_race_concept.concept_name as race,
        person.ethnicity_concept_id,
        p_ethnicity_concept.concept_name as ethnicity,
        person.sex_at_birth_concept_id,
        p_sex_at_birth_concept.concept_name as sex_at_birth 
    FROM
        `""" + os.environ["WORKSPACE_CDR"] + """.person` person 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_gender_concept 
            ON person.gender_concept_id = p_gender_concept.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_race_concept 
            ON person.race_concept_id = p_race_concept.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_ethnicity_concept 
            ON person.ethnicity_concept_id = p_ethnicity_concept.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_sex_at_birth_concept 
            ON person.sex_at_birth_concept_id = p_sex_at_birth_concept.concept_id"""

dataset_15423695_person_df = pd.read_gbq(
    dataset_15423695_person_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

demog_all = dataset_15423695_person_df[dataset_15423695_person_df['person_id'].isin([int(i) for i in sample_ids])]

Downloading:   0%|          | 0/413457 [00:00<?, ?rows/s]

In [13]:
# Convert date_of_birth to datetime
demog_all['date_of_birth'] = pd.to_datetime(demog_all['date_of_birth']).dt.tz_localize(None)

now = pd.Timestamp(datetime.now())

def timedelta_to_years(td):
    return td / np.timedelta64(1, 'Y')
# Calculate age
demog_all['age'] = (now - demog_all['date_of_birth']).apply(timedelta_to_years)

/tmp/ipykernel_6040/1777470070.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_6040/1777470070.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [15]:
demog_all = demog_all[['person_id','gender','age','race','ethnicity', 'date_of_birth']]
demog_all['person_id'] = demog_all['person_id'].astype(str)
# demog_all = demog_all.merge(ancestry_pred_df, on = 'person_id', how = 'inner')

sex_miss = demog_all[demog_all['gender'].isin(['PMI: Skip','Gender Identity: Additional Options','I prefer not to answer',
                                               'Not man only, not woman only, prefer not to answer, or skipped',
                                              'Gender Identity: Transgender','Gender Identity: Non Binary'])]['person_id'].tolist()
eth_miss = demog_all[demog_all['ethnicity'].isin(['PMI: Skip','What Race Ethnicity: Race Ethnicity None Of These',
                                          'PMI: Prefer Not To Answer'])]['person_id'].tolist()

rm_pat = list(set(sex_miss+eth_miss))
demog_all = demog_all[~demog_all['person_id'].isin(rm_pat)]

In [16]:
demog_all['person_id'] = demog_all['person_id'].astype(int)

In [17]:
mmr = list(set(mlh1_pat + msh2_pat + msh6_pat + pms2_pat))
control = list(set(list(demog_all['person_id']))-set(mmr))
print(len(mmr))
print(len(control))

682
217227


In [18]:
def demog_by_mmr(patid):
    mmr_df = pd.DataFrame(patid)
    mmr_df.columns = ['person_id']
    demog_mmr =mmr_df.merge(demog_all, on = 'person_id', how = 'inner')
    return demog_mmr

In [19]:
demog_all = demog_all.reset_index(drop = True)
mlh1_df = demog_by_mmr(mlh1_pat)
mlh1_df['group'] = 'MLH1'
msh2_df = demog_by_mmr(msh2_pat)
msh2_df['group'] = 'MSH2'
msh6_df = demog_by_mmr(msh6_pat)
msh6_df['group'] = 'MSH6'
pms2_df = demog_by_mmr(pms2_pat)
pms2_df['group'] = 'PMS2'
control_df = demog_by_mmr(control)
control_df['group'] = 'control'

In [21]:
grouped_demog = pd.concat([mlh1_df, msh2_df, msh6_df, pms2_df, control_df])

In [23]:
Counter(grouped_demog['group'])

Counter({'control': 217227, 'PMS2': 316, 'MSH6': 212, 'MLH1': 63, 'MSH2': 55})

In [24]:
316+212+63+55

646

In [29]:
217227-13-5

217209

### remove pms2 psuedo and wrong sex

In [25]:
bucket = os.getenv("WORKSPACE_BUCKET")
!gsutil ls $WORKSPACE_BUCKET/data
pms2_pseudo =pd.read_csv(f'{bucket}/data/pms2_cl_pats.csv')

print(len(pms2_pseudo['pat'].unique()))
remove = [2954789, 1638459, 1712944, 1939350, 2163757] # patients with wrong sex (reported as Male but having endometrial cancer)
remove = remove+list(pms2_pseudo['pat'].unique())

gs://fc-secure-9b1ab35f-6336-4ab5-aadc-2d39277e3d9b/data/LS_fam.csv
gs://fc-secure-9b1ab35f-6336-4ab5-aadc-2d39277e3d9b/data/Lynch_related_var.tsv
gs://fc-secure-9b1ab35f-6336-4ab5-aadc-2d39277e3d9b/data/Lynch_var.tsv
gs://fc-secure-9b1ab35f-6336-4ab5-aadc-2d39277e3d9b/data/acaf_X_filt.bed
gs://fc-secure-9b1ab35f-6336-4ab5-aadc-2d39277e3d9b/data/acaf_X_filt.bim
gs://fc-secure-9b1ab35f-6336-4ab5-aadc-2d39277e3d9b/data/acaf_X_filt.fam
gs://fc-secure-9b1ab35f-6336-4ab5-aadc-2d39277e3d9b/data/acaf_X_filt.log
gs://fc-secure-9b1ab35f-6336-4ab5-aadc-2d39277e3d9b/data/acaf_X_filt.nosex
gs://fc-secure-9b1ab35f-6336-4ab5-aadc-2d39277e3d9b/data/acaf_Y_filt.bed
gs://fc-secure-9b1ab35f-6336-4ab5-aadc-2d39277e3d9b/data/acaf_Y_filt.bim
gs://fc-secure-9b1ab35f-6336-4ab5-aadc-2d39277e3d9b/data/acaf_Y_filt.fam
gs://fc-secure-9b1ab35f-6336-4ab5-aadc-2d39277e3d9b/data/acaf_Y_filt.log
gs://fc-secure-9b1ab35f-6336-4ab5-aadc-2d39277e3d9b/data/acaf_Y_filt.nosex
gs://fc-secure-9b1ab35f-6336-4ab5-aadc-2d39277e3

In [26]:
rev_pat = list(set(list(grouped_demog['person_id'].unique())) - set(remove))
rev_pat = pd.DataFrame(rev_pat)
rev_pat.columns = ['person_id'] 
grouped_demog = grouped_demog.merge(rev_pat, on = 'person_id', how = 'inner')

In [27]:
Counter(grouped_demog['group'])

Counter({'control': 217209, 'PMS2': 286, 'MSH6': 212, 'MLH1': 63, 'MSH2': 55})

In [5]:
save_path = f'{bucket}/data/aou_demog.csv'
grouped_demog.to_csv(save_path, index = None)

In [6]:
def median_with_ci(data, num_bootstrap=1000, ci=95):
    medians = []
    n = len(data)
    
    # Generate bootstrap samples and compute medians
    for _ in range(num_bootstrap):
        sample = np.random.choice(data, size=n, replace=True)
        medians.append(np.median(sample))
    
    # Calculate the median and the confidence interval
    median = np.median(data)
    lower_bound = np.percentile(medians, (100 - ci) / 2)
    upper_bound = np.percentile(medians, 100 - (100 - ci) / 2)
    
    return median, lower_bound, upper_bound

In [7]:
demog_case = grouped_demog[grouped_demog['group']!='control']
demog_ctrl = grouped_demog[grouped_demog['group']=='control']

In [8]:
# Calculate the median and 95% CI for the 'age' column
median_age, ci_lower, ci_upper = median_with_ci(demog_case['age'])

print(f"Median age: {median_age}")
print(f"95% CI: [{ci_lower}, {ci_upper}]")

Median age: 58.04058655540751
95% CI: [56.03784682938011, 60.04058655540751]


In [9]:
# Calculate the median and 95% CI for the 'age' column
median_age, ci_lower, ci_upper = median_with_ci(demog_ctrl['age'])

print(f"Median age: {median_age}")
print(f"95% CI: [{ci_lower}, {ci_upper}]")

Median age: 59.04058655540751
95% CI: [58.04058655540751, 59.04058655540751]


In [10]:
# Calculate the median and 95% CI for the 'age' column
median_age, ci_lower, ci_upper = median_with_ci(grouped_demog['age'])

print(f"Median age: {median_age}")
print(f"95% CI: [{ci_lower}, {ci_upper}]")

Median age: 59.04058655540751
95% CI: [58.04058655540751, 59.04058655540751]
